## Datenalyse: 04 Datenaufbereitung für EVT

Das Ziel ist es, die Wahrscheinlichkeit einer X45 Sonneneruption zu berechnen.

X45: 0.0045000000000000005

### Aufbereitung der Daten (nur ein Wert pro Tag)

Code auskommentiert, da die CSV-Datei nicht jedes Mal neu erstellt werden muss.

In folgendem Code wird folgendes gemacht:

Ziel: eine CSV-Datei mit dem richtigen Format für die Anwendung von pyextremes erstellen<br>
Step 1: Aufbereitete Daten in das Dataframe "ds_sf" importieren (nur Spalten 'date' und 'XRay_class_nr1')<br>
Step 2: Spalte 'date' (Datum) als Index setzen<br>

Zwischen-Info: Da das Dataframe dt_temp aktuell nicht für jeden Tag Werte hat und für manche Tage mehrere Werte, muss das Dataframe umgewandelt werden, sodass für jeden Tag ein Wert und pro Tag nur ein Wert existiert.<br>
-> Warum darf es nicht mehrere Werte pro Tag haben?? -> deshalb neue Aufbereitung


Step 3: Erstelle ein neues Dataframe mit einem Datums-Index von Anfang bis Ende des Original-Dataframes<br>
Step 4: Füge die Spalte "Sunspots" hinzu und fülle sie mit 0 (-> alle Tage ohne Wert erhalten neu 0 als Wert)<br>
Step 5: Schleife durch die Daten im Original-Dataframe und fülle das neue Dataframe jeweils mit dem höchstem Wert pro Tag<br>
Step 6: Speichere das neue Dataframe als "df_extreme.csv ab"<br>

In [ ]:
# Ziel: eine CSV-Datei mit dem richtigen Format für die Anwendung von pyextremes erstellen
# tutorial: https://georgebv.github.io/pyextremes/user-guide/3-block-maxima/

# Importe
import pandas as pd

# Aufbereitete Daten in das Dataframe "ds_sf" importieren (nur Spalten 'date' und 'XRay_class_nr1')
df_temp = pd.read_csv('new_data_solar_flare/solar_flare_prepared.csv', sep=';' ,usecols=['date', 'XRay_class_nr1'])

# Spalte 'date' (Datum) als Index setzen
df_temp.set_index('date', inplace=True)

# Da das Dataframe dt_temp aktuell nicht für jeden Tag Werte hat und für manche Tage mehrere Werte,
# muss das Dataframe umgewandelt werden, sodass für jeden Tag ein Wert und pro Tag nur ein Wert existiert.

# Erstelle ein neues Dataframe mit einem Datums-Index von Anfang bis Ende des Original-Dataframes
date_range = pd.date_range(start=df_temp.index.min(), end=df_temp.index.max(), freq='D')
df_new = pd.DataFrame(index=date_range)

# Füge die Spalte "Sunspots" hinzu und fülle sie mit 0 (-> alle Tage ohne Wert erhalten neu 0 als Wert)
df_new['XRay_class_nr1'] = 0

# Schleife durch die Daten im Original-Dataframe und fülle das neue Dataframe entsprechend
for index, row in df_temp.iterrows():
    # Wenn ein Eintrag für diesen Tag im Original-Dataframe existiert, füge den höchsten Wert hinzu
    if index in df_new.index:
        max_sf = df_temp.loc[index]['XRay_class_nr1'].max()
        df_new.at[index, 'XRay_class_nr1'] = max_sf

# Speichere das neue Dataframe als "df_extreme.csv ab"
df_new.to_csv('new_data_solar_flare/df_extreme.csv')

In [ ]:
import pandas as pd

df = pd.read_csv('new_data_solar_flare/df_extreme.csv', sep=',', index_col=0)

df.describe()

### Aufbereitung für R

In [1]:
#Speichern inkl. 0-Werte und mit einem Wert pro Tag (distinct)

import pandas as pd

df_R = pd.read_csv('new_data_solar_flare/df_extreme.csv', sep=',',usecols=['XRay_class_nr1'])

# in Datei df_extreme_neu.csv speichern
df_R.to_csv('new_data_solar_flare/R_df_solarflare_distinct.csv', index=False, header=False)

In [12]:
#Speichern inkl. 0-Werte
import pandas as pd

df_R = pd.read_csv('new_data_solar_flare/df_extreme_neu.csv', sep=',',usecols=['XRay_class_nr1'])

# in Datei df_extreme_neu.csv speichern
df_R.to_csv('new_data_solar_flare/R_df_solarflare_new.csv', index=False, header=False)

In [13]:
#Speichern ohne 0-Werte
import pandas as pd

df_R_wtout_zero = pd.read_csv('new_data_solar_flare/df_extreme_neu.csv', sep=',',usecols=['XRay_class_nr1'])

df_R_wtout_zero = df_R_wtout_zero[df_R_wtout_zero['XRay_class_nr1'] > 0]

# in Datei df_extreme_neu.csv speichern
df_R_wtout_zero.to_csv('new_data_solar_flare/R_df_solarflare_wtout_zero.csv', index=False, header=False)

### Aufbereitung der Daten (1 bis n Werte pro Tag)

Code auskommentiert, da die CSV-Datei nicht jedes Mal neu erstellt werden muss.

Ausgangslage:
Da das aufbereitete CSV "solar_flare_prepared.csv" aktuell nicht für jeden Tag Werte hat, müssen die fehlenden Tage noch ergänzt werden.

Das CSV "df_extreme_test.csv" ist ein Dataframe, welches die gleichen Daten (plural von Datum) hat wie die aufbereiteten Daten,
nur dass die fehlenden Daten bereits eingefügt wurden. Allerdings fehlen da noch die Werte der Spale "XRay_class_nr1". Deshalb werden die beiden Dataframes hier zusammengemerged.<br>
Achtung: Den Code für die Erstellung von "df_extreme_test.csv" habe ich nicht mehr - deshalb auf keinen Fall löschen!!!

Ziel: eine CSV-Datei mit dem richtigen Format für die Anwendung von pyextremes erstellen<br>

Step 1: vorbereitetes CSV "df_extreme_test.csv" in Dataframe "df_val" importieren<br>
Step 2: Aufbereitete Daten in das Dataframe "df_val" importieren (nur Spalten 'date' und 'XRay_class_nr1')<br>
Step 3: Die beiden Dataframes mergen<br>
Step 4: Alle Reihen entfernen in welchen der Wert in der Spalte "XRay_class_nr1" = 0 ist und bereits andere Einträge für diesen Tag existieren. Wenn es keine Einträge an diesem Tag gibt, ist der Wert 0 in Ordnung.<br>
Step 5: Speichere das neue Dataframe als "df_extreme.csv ab"<br>

In [ ]:
import numpy as np
import pandas as pd

df_idx = pd.read_csv('new_data_solar_flare/df_extreme_test.csv', sep=',', usecols=['date'])
df_val = pd.read_csv('new_data_solar_flare/solar_flare_prepared.csv', sep=';' ,usecols=['date', 'XRay_class_nr1'])

result = pd.concat([df_idx, df_val], axis=0)

# Ersetze NAN durch 9
result.replace(np. nan,0, inplace=True)


result.sort_values('date', inplace=True)

# Überprüfung der Häufigkeit von Werten in der Spalte "eins"
counts = result['date'].value_counts()

# Wenn die Spalte "XRay_class_nr1" = 0 ist und das Datum nochmals vorkommt, als "delete" markieren, sonst als "keep"
result['test'] = np.where((result['XRay_class_nr1'] == 0) & 
                          (result['date'].apply(lambda x: True if counts[x] > 1 else False)), 'delete', 'keep')

# Nur die Daten behalten, die als "keep" markiert sind, behalten
# und nur die Spalten "date" und "XRay_class_nr1" behalten
result = result[result['test'] == 'keep'][['date', 'XRay_class_nr1']]

result.set_index('date', inplace=True)

# in Datei df_extreme_neu.csv speichern
result.to_csv('new_data_solar_flare/df_extreme_neu.csv')


#result.head(10)
#result.drop_duplicates()